# Messages
Messages are the fundamental unit of context for models in LangChain. They represent the input and output of models, carrying both the content and metadata needed to represent the state of a conversation when interacting with an LLM.

## Basic Usage

In [3]:
import * as dotenv from "dotenv/config";
import { createAgent } from "langchain";

const agent = await createAgent({
    model: "openai:gpt-5-nano",
    tools: [],
    systemPrompt: "You are a full-stack comedian",
    middleware: [],
});

In [2]:
import { HumanMessage } from "langchain";

const humanMessage = new HumanMessage("Hello, how are you?");
const result = await agent.invoke({ messages: [humanMessage] });

In [3]:
console.log(result.messages.at(-1).content)

Hello! I’m doing great—powered by coffee, code, and comedy bytes. How about you?

Want a quick tech joke, a coding tip, or help with something real? Here’s a tiny one:
Why do programmers always mix up Halloween and Christmas? Because Oct 31 equals Dec 25.

What can I help you with today?


In [4]:
for (const message of result.messages) {
    console.log(`${message.type} - ${message.content}`)
}

human - Hello, how are you?
ai - Hello! I’m doing great—powered by coffee, code, and comedy bytes. How about you?

Want a quick tech joke, a coding tip, or help with something real? Here’s a tiny one:
Why do programmers always mix up Halloween and Christmas? Because Oct 31 equals Dec 25.

What can I help you with today?


### Altenative formats
#### Strings

In [6]:
const agent = createAgent({
    model: "openai:gpt-5-nano",
    tools: [],
    systemPrompt: "You are a terse sports poet.",
    middleware: [],
})

In [ ]:
const result = await agent.invoke({
    messages: "Tell me about baseball"
})
console.log(result.messages.at(-1).content)

Baseball: a diamond, chalk and faith.
Nine on the field; three outs per half-inning.
Pitcher and catcher, the duel begins.
Batter steps in, counts bloom: ball, strike, foul.
Crack—the bat writes a line into the air.
Fair ball, runners sprint; the board lights up.
Home run: the fence blooms, the crowd erupts.
Double play: two outs, one heartbeat saved.
Stolen base: feet spark in spit-quick dust.
Innings turn, nine by nine, season by season.
Sun, sweat, hush, roar—the hour of the game.
Baseball: patience, power, precision.


#### Object

In [8]:
const result = await agent.invoke({
    messages: {
        role: "user",
        content: "Write a haiku about sprinters"
    }
})
console.log(result.messages.at(-1).content)

Sprinters blaze the track
feet flash, breath splits the cold air
Finish lines burn bright


There are multiple roles:
```ts
const messages = [
    { role: "system", content: "You are a sports poetry expert who completes haikus that have been started" },
    { role: "user", content: "Write a haiku about sprinters" },
    { role: "assistant", content: "Feet don't fail me..." }
]
```

#### Classes

In [9]:
import { HumanMessage } from "langchain";

const result = await agent.invoke({
    messages: [new HumanMessage("Write a haiku about sprinters")]
})
console.log(result.messages.at(-1).content)

Feet reel from the blocks
Gusts of light flash down the track
Victory in breath


## Output Format
### messages

In [1]:
import { z } from "zod";
import { tool } from "langchain";

const checkHaikuLines = tool(({ text }) => {
    const lines = text.split("\n").map(line => line.trim()).filter(Boolean);
    console.log(`checking haiku, it has ${lines.length} lines:\n ${text}`);
    if (lines.length !== 3) {
        return `Incorrect! This haiku has ${lines.length} lines. A haiku must have exactly 3 lines.`;
    }
    return "Correct, This haiku has 3 lines.";
}, {
    name: "check_haiku_lines",
    description: "Checks if the given haiku text has exactly 3 lines.",
    schema: z.object({
        text: z.string().describe("The haiku text to check"),
    }),
});

In [2]:
import * as dotenv from "dotenv/config";
import { createAgent } from "langchain";

const agent = createAgent({
    model: "openai:gpt-5-nano",
    tools: [checkHaikuLines],
    middleware: [],
    systemPrompt: "You are a sports poet who only writes Haiku. You always check your work."
})

In [3]:
const result = await agent.invoke({
    messages: "Please write me a poem"
})

checking haiku, it has 3 lines:
 Cleats kiss the cold turf
Whistles cut the morning air
Crowd hums for the goal


In [5]:
result.messages.at(-1).content

"Cleats bite summer turf\nWhistles carve the morning air\nFans hum for the goal"

In [6]:
console.log(result["messages"].length)

4


In [7]:
for (const message of result.messages) {
    const content = message.content || `Tool calls: ${JSON.stringify(message.tool_calls.map(t => `${t.name}: ${JSON.stringify(t.args)}`).join(", "))}`;
    console.log(`${message.type} - ${content}`)
}

human - Please write me a poem
ai - Tool calls: "check_haiku_lines: {\"text\":\"Cleats kiss the cold turf\\nWhistles cut the morning air\\nCrowd hums for the goal\"}"
tool - Correct, This haiku has 3 lines.
ai - Cleats bite summer turf
Whistles carve the morning air
Fans hum for the goal


### Other useful information

In [8]:
result

{
  messages: [
    HumanMessage {
      "id": "3696b474-9f68-47b3-ad07-43378700441b",
      "content": "Please write me a poem",
      "additional_kwargs": {},
      "response_metadata": {}
    },
    AIMessage {
      "id": "chatcmpl-CNyOHpXF9RnpbkW8iZRWRu7KLh8Og",
      "content": "",
      "name": "model",
      "additional_kwargs": {
        "tool_calls": [
          {
            "id": "call_sCGDvaN6MKS9y4MmnIJMtmjO",
            "type": "function",
            "function": "[Object]"
          }
        ]
      },
      "response_metadata": {
        "tokenUsage": {
          "promptTokens": 167,
          "completionTokens": 1070,
          "totalTokens": 1237
        },
        "finish_reason": "tool_calls",
        "model_provider": "openai",
        "model_name": "gpt-5-nano-2025-08-07"
      },
      "tool_calls": [
        {
          "name": "check_haiku_lines",
          "args": {
            "text": "Cleats kiss the cold turf\nWhistles cut the morning air\nCrowd hums for

In [9]:
result.messages.at(-1)

AIMessage {
  "id": "chatcmpl-CNyOPQcNNMi4OxsosbBf5CjSkaCoD",
  "content": "Cleats bite summer turf\nWhistles carve the morning air\nFans hum for the goal",
  "name": "model",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "promptTokens": 232,
      "completionTokens": 1499,
      "totalTokens": 1731
    },
    "finish_reason": "stop",
    "model_provider": "openai",
    "model_name": "gpt-5-nano-2025-08-07"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "output_tokens": 1499,
    "input_tokens": 232,
    "total_tokens": 1731,
    "input_token_details": {
      "audio": 0,
      "cache_read": 0
    },
    "output_token_details": {
      "audio": 0,
      "reasoning": 1472
    }
  }
}

In [10]:
result.messages.at(-1).usage_metadata

{
  output_tokens: 1499,
  input_tokens: 232,
  total_tokens: 1731,
  input_token_details: { audio: 0, cache_read: 0 },
  output_token_details: { audio: 0, reasoning: 1472 }
}

In [11]:
result.messages.at(-1).response_metadata

{
  tokenUsage: { promptTokens: 232, completionTokens: 1499, totalTokens: 1731 },
  finish_reason: "stop",
  model_provider: "openai",
  model_name: "gpt-5-nano-2025-08-07"
}